<a href="https://colab.research.google.com/github/AfsanehHabibi/reddit-conversation-quality/blob/main/Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base_path = "/content/drive/MyDrive/University/RedditData/"

In [ ]:
import pickle

with open(f"{base_path}conversations_latest_dataframe.pkl", 'rb') as file:
    df = pickle.load(file)

In [ ]:
df.head()

,id,conversation_id,reply_to,readability,evidence,has_reasoning,has_reasoning_one_shot,politeness,relatedness,subjectivity,...,mean_norm_readability_replies,mean_norm_politeness_replies,mean_relatedness_replies,mean_binary_evidence_replies,mean_has_reasoning_replies,len_title,len_text,log_subreddit_subscribers,depth,merged_depth
0,xyt38s,xyt38s,None,-0.5,0.5,0,0.785905,0.095238,NaN,0.532381,...,0.65,0.058824,0.306438,0.666667,0.0,12.0,280,8.821732,0,0
1,iromk5a,xyt38s,xyt38s,-0.5,0.5,0,0.725411,0.095238,0.887727,0.532381,...,NaN,NaN,NaN,NaN,NaN,NaN,276,8.821732,1,1
2,irpc624,xyt38s,xyt38s,-4.5,0.5,0,0.882337,0.000000,0.031587,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,60,8.821732,1,1
3,irp3pvs,xyt38s,xyt38s,-5.5,0.0,0,0.968893,0.047619,0.000000,0.400000,...,NaN,NaN,NaN,NaN,NaN,NaN,19,8.821732,1,1
4,xyt38b,xyt38b,None,-2.5,0.5,1,0.390972,0.380952,NaN,0.415179,...,1.00,0.294118,0.262232,1.000000,1.0,66.0,694,14.904485,0,0


In [ ]:
df.columns

Index(['id', 'conversation_id', 'reply_to', 'readability', 'evidence',
       'has_reasoning', 'has_reasoning_one_shot', 'politeness', 'relatedness',
       'subjectivity', 'polarity', 'text', 'new_readability',
       'norm_readability', 'norm_politeness', 'title', 'subreddit',
       'subreddit_subscribers', 'topic', 'binary_evidence',
       'comments_count_conv', 'mean_norm_readability_conv',
       'mean_norm_politeness_conv', 'mean_relatedness_conv',
       'mean_binary_evidence_conv', 'mean_has_reasoning_conv',
       'mean_norm_readability_replies', 'mean_norm_politeness_replies',
       'mean_relatedness_replies', 'mean_binary_evidence_replies',
       'mean_has_reasoning_replies', 'len_title', 'len_text',
       'log_subreddit_subscribers', 'depth', 'merged_depth'],
      dtype='object')

In [ ]:
features_to_use = ['has_reasoning',
       'subjectivity', 'polarity', 'norm_readability', 'norm_politeness',
       'subreddit_subscribers', 'binary_evidence', 'len_text', 'depth']
       ##topic , 'subreddit','relatedness','len_title',

In [ ]:
features_to_predict = ['mean_norm_readability_replies', 'mean_norm_politeness_replies',
       'mean_relatedness_replies', 'mean_binary_evidence_replies',
       'mean_has_reasoning_replies']

In [ ]:
len(df_clean)

99082

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# There is less than 100 rows with NaN value in subreddit_subscribers
df_clean = df.dropna(subset=['subreddit_subscribers'])
# Drop rows where any of the features_to_predict are NaN
df_clean = df_clean.dropna(subset=features_to_predict)

# Prepare the data
X = df_clean[features_to_use]
y = df_clean[features_to_predict]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Make predictions
y_pred = rf.predict(X_test)

# Evaluate the model for each target feature
for i, target in enumerate(features_to_predict):
    print(f'Evaluating for {target}:')
    mse = mean_squared_error(y_test.iloc[:, i], y_pred[:, i])
    r2 = r2_score(y_test.iloc[:, i], y_pred[:, i])
    print(f'Mean Squared Error: {mse}')
    print(f'R^2 Score: {r2}')
    print('---')

Evaluating for mean_norm_readability_replies:
Mean Squared Error: 0.04378170995298247
R^2 Score: 0.027988145428755273
---
Evaluating for mean_norm_politeness_replies:
Mean Squared Error: 0.014513799982016554
R^2 Score: 0.15816924134676258
---
Evaluating for mean_relatedness_replies:
Mean Squared Error: 0.010575955728239612
R^2 Score: 0.15339442338036002
---
Evaluating for mean_binary_evidence_replies:
Mean Squared Error: 0.15153117115869918
R^2 Score: -0.009476555800185027
---
Evaluating for mean_has_reasoning_replies:
Mean Squared Error: 0.14633031051610884
R^2 Score: -0.0052864052048609
---


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Define your features and targets
text_feature = 'text'  # replace with your actual text column name

# Replace NaN values in 'subreddit_subscribers' with the column's average
if 'subreddit_subscribers' in df.columns:
    avg_subscribers = df['subreddit_subscribers'].mean()
    df['subreddit_subscribers'] = df['subreddit_subscribers'].fillna(avg_subscribers)

# Drop rows where any of the features_to_predict are NaN
df_clean = df.dropna(subset=features_to_predict)

# Prepare the data
X = df_clean[features_to_use + [text_feature]]
y = df_clean[features_to_predict]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a column transformer to handle both numeric and text features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), features_to_use),
        ('text', TfidfVectorizer(), text_feature)
    ]
)

# Create a pipeline that first preprocesses the data and then applies the model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Train the model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model for each target feature
for i, target in enumerate(features_to_predict):
    print(f'Evaluating for {target}:')
    mse = mean_squared_error(y_test.iloc[:, i], y_pred[:, i])
    r2 = r2_score(y_test.iloc[:, i], y_pred[:, i])
    print(f'Mean Squared Error: {mse}')
    print(f'R^2 Score: {r2}')
    print('---')


ImportError: cannot import name 'make_column_transformer' from 'sklearn.pipeline' (/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Define your features and targets
text_feature = 'text'  # replace with your actual text column name

# Replace NaN values in 'subreddit_subscribers' with the column's average
if 'subreddit_subscribers' in df.columns:
    avg_subscribers = df['subreddit_subscribers'].mean()
    df['subreddit_subscribers'] = df['subreddit_subscribers'].fillna(avg_subscribers)

# Drop rows where any of the features_to_predict are NaN
df_clean = df.dropna(subset=features_to_predict)

# Prepare the data
X = df_clean[features_to_use + [text_feature]]
y = df_clean[features_to_predict]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a column transformer to handle both numeric and text features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), features_to_use),
        ('text', TfidfVectorizer(), text_feature)
    ]
)

# Create a pipeline that first preprocesses the data and then applies the model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Train the model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model for each target feature
for i, target in enumerate(features_to_predict):
    print(f'Evaluating for {target}:')
    mse = mean_squared_error(y_test.iloc[:, i], y_pred[:, i])
    r2 = r2_score(y_test.iloc[:, i], y_pred[:, i])
    print(f'Mean Squared Error: {mse}')
    print(f'R^2 Score: {r2}')
    print('---')
